## ***Kindly make a copy of this notebook and start experimenting.***

## Clone the github repo

In [1]:
!git clone https://github.com/sgandhi31/LPRNet_Optimization.git
%cd LPRNet_Optimization

# Check files and structure
!ls


Cloning into 'LPRNet_Optimization'...
remote: Enumerating objects: 1116, done.
remote: Counting objects: 100% (1116/1116), done.
remote: Compressing objects: 100% (1108/1108), done.
remote: Total 1116 (delta 46), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1116/1116), 4.09 MiB | 12.03 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/LPRNet_Optimization
data	 mlc	README.md	  test_LPRNetOpt_mlc.py  test_LPRNet_quantize.py  tuning_log.json
LICENSE  model	requirements.txt  test_LPRNet.py	 train_LPRNet.py	  weights


In [2]:
# Install necessary Python libraries
!pip install torch torchvision opencv-python-headless imutils pillow numpy


In [3]:
# Check PyTorch and GPU setup
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")


PyTorch Version: 2.5.1+cu121
GPU Available: False


## Test the original model

In [11]:
# Run the test script to verify setup

!python test_LPRNet.py --test_img_dirs 'data/test' --pretrained_model 'weights/Final_LPRNet_model.pth'


Successful to build network!
/content/LPRNet_Optimization/LPRNet_Optimization/LPRNet_Optimization/test_LPRNet.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.loa

### Get model size of original model

In [ ]:
!pip install onnxruntime onnxruntime-tools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
import torch
from model.LPRNet import build_lprnet

# Load pretrained model
model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
model.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 3, 24, 94)  # Input size for LPRNet
torch.onnx.export(model, dummy_input, "/content/LPRNet_Optimization/LPRNet.onnx", verbose=True)
print("Model exported to ONNX format.")


<ipython-input-5-e1b03eec9997>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Final_LPRNet_model.pth

Model exported to ONNX format.


In [ ]:
# Check ONNX file size
import os
onnx_file = "/content/LPRNet_Optimization/LPRNet.onnx"
size_mb = os.path.getsize(onnx_file) / (1024 * 1024)
print(f"ONNX Model Size: {size_mb:.2f} MB")


ONNX Model Size: 1.91 MB


#### Original Model:

1. Size: 1.91 MB

2. Test Accuracy: 90.3%

3. Inference Time: 0.2 seconds

## Model Optimization 1 - PTQ


In [ ]:
#test the script

!python test_LPRNet_quantize.py --test_img_dirs 'data/test' --quantized_model 'weights/Final_LPRNet_model.pth'


/content/LPRNet_Optimization/test_LPRNet_quantize.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device

In [ ]:
size_mb = os.path.getsize("/content/LPRNet_Optimization/weights/Quantized_LPRNet_model.pth") / (1024 * 1024)
print(f"Quantized Model Size: {size_mb:.2f} MB")

Quantized Model Size: 0.51 MB


#### Optimized Model (Post Training Quantization):

1. Size: 0.51 MB

2. Test Accuracy: 78%

3. Inference Time: 0.027 Seconds


In [ ]:
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 10.5 MB/s eta 0:00:00


## Run Quantized model Test In Notebook

In [17]:
import torch
import torch.nn as nn
import torch.quantization
from torch.ao.quantization import QuantStub, DeQuantStub

# Define a wrapper to add QuantStub and DeQuantStub
class QuantizedLPRNet(nn.Module):
    def __init__(self, model):
        super(QuantizedLPRNet, self).__init__()
        self.quant = QuantStub()  # Handles input quantization
        self.model = model
        self.dequant = DeQuantStub()  # Handles output dequantization

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

def load_pretrained_model(model_path):
    # Load the pretrained LPRNet model
    model = build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def apply_quantization(model, calibration_data):
    # Wrap the model with QuantStub and DeQuantStub
    model = QuantizedLPRNet(model)

    # Define the quantization configuration
    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

    # Prepare the model for quantization
    model_prepared = torch.ao.quantization.prepare(model, inplace = False)

    # Calibrate the model using a representative dataset
    model_prepared.eval()
    with torch.no_grad():
      model_prepared(calibration_data)

    # Convert to quantized version
    model_quantized = torch.ao.quantization.convert(model_prepared, inplace = False)
    return model_quantized


def save_pth_model(model_path, save_path):
  # # Load your trained model
  model_path = 'weights/Final_LPRNet_model.pth'
  model = load_pretrained_model(model_path)

  # # Example calibration data (replace with your actual calibration data)
  calibration_data = torch.randn(1, 3, 24, 94) # Dummy data

  # # Apply quantization
  quantized_model = apply_quantization(model, calibration_data)

  # # Save the quantized model
  torch.save(quantized_model.state_dict(), savepath)

def get_model_size(model_path):
  size_mb = os.path.getsize(model_path) / (1024 * 1024)
  print(f"Model Size: {size_mb:.2f} MB")



# print("Quantization applied and model saved.")


In [ ]:
import time
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
# from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os



def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
datasets = LPRDataLoader(test_img_dirs.split(','), [94, 24], 8)

epoch_size = len(datasets) // 100
batch_iterator = iter(DataLoader(datasets, 100, shuffle=True, num_workers=8, collate_fn=collate_fn))
#calibration:
model_path = 'weights/Final_LPRNet_model.pth'
new_model = load_pretrained_model(model_path)
calibration_data,_,_ = next(iter(DataLoader(datasets,100, shuffle = True, collate_fn = collate_fn)))
quantized_model = apply_quantization(new_model, calibration_data)


Tp = 0
Tn_1 = 0
Tn_2 = 0
t1 = time.time()
for i in range(epoch_size):
    # load train data
    images, labels, lengths = next(batch_iterator)
    start = 0
    targets = []
    for length in lengths:
        label = labels[start:start+length]
        targets.append(label)
        start += length
    targets = np.array([el.numpy() for el in targets])
    imgs = images.numpy().copy()

    images = Variable(images)
    # new_model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
    # calibration_data = torch.randn(1, 3, 24, 94) # Dummy data
    # quantized_model = apply_quantization(new_model, calibration_data)

    # quantized_model = QuantizedLPRNet(new_model)
    # quantized_model.load_state_dict(torch.load("Quantized_LPRNet_model.pth", map_location = torch.device('cpu')))
    # Apply quantization

    prebs = quantized_model(images)
    # greedy decode
    prebs = prebs.cpu().detach().numpy()
    preb_labels = list()
    for i in range(prebs.shape[0]):
        preb = prebs[i, :, :]
        preb_label = list()
        for j in range(preb.shape[1]):
            preb_label.append(np.argmax(preb[:, j], axis=0))
        no_repeat_blank_label = list()
        pre_c = preb_label[0]
        if pre_c != len(CHARS) - 1:
            no_repeat_blank_label.append(pre_c)
        for c in preb_label: # dropout repeate label and blank label
            if (pre_c == c) or (c == len(CHARS) - 1):
                if c == len(CHARS) - 1:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        preb_labels.append(no_repeat_blank_label)
    for i, label in enumerate(preb_labels):
        if len(label) != len(targets[i]):
            Tn_1 += 1
            continue
        if (np.asarray(targets[i]) == np.asarray(label)).all():
            Tp += 1
        else:
            Tn_2 += 1
Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
t2 = time.time()
print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-102-f9d4bd70b59e>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary

[Info] Test Accuracy: 0.78 [780:129:91:1000]
[Info] Test Speed: 0.023357553005218506s 1/1000]


In [ ]:
torch.save(quantized_model.state_dict(), 'Quantized_LPRNet_model.pth')

print("Quantization applied and model saved.")

Quantization applied and model saved.


## Model Optimization - Unstrcutured Pruning

#### Adjust the pruning ratio to experiement with different versions of pruned models.

In [22]:
!python3 model/LPRNet_pruned.py --pretrained_model '/content/LPRNet_Optimization/weights/Final_LPRNet_model.pth' --pruning_ratio 0.5 --verbose True

/content/LPRNet_Optimization/model/LPRNet_pruned.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu

#### Test the pruned model

In [13]:
!python test_LPRNet.py --test_img_dirs 'data/test' --pretrained_model '/content/LPRNet_Optimization/weights/unstructured_pruned_LPRNet_model_0.5.pth'

Successful to build network!
/content/LPRNet_Optimization/LPRNet_Optimization/LPRNet_Optimization/test_LPRNet.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.loa

#### Pruned model:

1. size : 1.91 MB
2. Accuracy: 82.5%
3. Inference Time: 0.15 seconds


## Pruned + Quantized

In [14]:
# Run the test script to verify setup

!python test_LPRNet_quantize.py --test_img_dirs 'data/test' --quantized_model '/content/LPRNet_Optimization/weights/unstructured_pruned_LPRNet_model_0.5.pth'


/content/LPRNet_Optimization/LPRNet_Optimization/LPRNet_Optimization/test_LPRNet_quantize.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.lo

In [27]:
def load_pretrained_model(modelObj, model_path):
    # Load the pretrained LPRNet model
    modelObj.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    modelObj.eval()
    return modelObj

def save_pth_model(model,model_path, save_path):
  model = load_pretrained_model(model, model_path)
  # # Example calibration data (replace with your actual calibration data)
  calibration_data = torch.randn(1, 3, 24, 94) # Dummy data

  # # Apply quantization
  quantized_model = apply_quantization(model, calibration_data)

  # # Save the quantized model
  torch.save(quantized_model.state_dict(), save_path)

def get_model_size(model_path):
  size_mb = os.path.getsize(model_path) / (1024 * 1024)
  print(f"Model Size: {size_mb:.2f} MB")

In [29]:
from model.LPRNet_quantize import build_lprnet_quantize
import os

model = build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
save_pth_model(model, '/content/LPRNet_Optimization/weights/unstructured_pruned_LPRNet_model_0.5.pth', 'weights/pruned_quantized_LPRNet_model.pth')
get_model_size('weights/pruned_quantized_LPRNet_model.pth')

<ipython-input-27-c03a5d0fa733>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelObj.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
/usr/loca

Model Size: 0.51 MB


## MLC optimization

In [24]:
!python3 -m  pip install mlc-ai-cpu -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 MB 5.6 MB/s eta 0:00:00


## Autotuning

## Directly check the optimized model

In [ ]:
# Run the test script to verify setup

!python test_LPRNetOpt_mlc.py --test_img_dirs 'data/test' --optimized_model 'weights/Final_LPRNet_model.pth' --log_file '/content/LPRNet_Pytorch/tuning_log.json'


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Compile...
/content/LPRNet_Optimization/mlc/load_autotune.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed d

## Pruned + MLC

In [32]:
# Run the test script to verify setup

!python test_LPRNetOpt_mlc.py --test_img_dirs 'data/test' --optimized_model '/content/LPRNet_Optimization/weights/unstructured_pruned_LPRNet_model_0.5.pth' --log_file '/content/LPRNet_Pytorch/tuning_log.json'


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Compile...
/content/LPRNet_Optimization/LPRNet_Optimization/LPRNet_Optimization/mlc/load_autotune.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limit

## Create AutoScheduling from scratch

In [26]:
import torch
import tvm
from tvm import relay, autotvm, auto_scheduler
from tvm.contrib import graph_executor, relay_viz
from model.LPRNet import build_lprnet


lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
lprnet.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
lprnet.eval()

#quantized model
# lprnet_quantize = build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
# lprnet_quantize.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Quantized_LPRNet_model.pth', map_location = torch.device('cpu')))
# lprnet_quantize.eval()
# lprnet = quantized_model
# model_path = 'weights/Final_LPRNet_model.pth'
# model = load_pretrained_model(model_path)
# model = QuantizedLPRNet(model)
# # Define the quantization configuration
# model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# # Prepare the model for quantization
# model_prepared = torch.ao.quantization.prepare(model, inplace = False)

# # Calibrate the model using a representative dataset
# model_prepared.eval()
# # Convert to quantized version
# model_quantized = torch.ao.quantization.convert(model_prepared, inplace = False)
# model_quantized.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Quantized_LPRNet_model.pth', map_location = torch.device('cpu')))

# Define an example input
input_shape = (1, 3, 24, 94)
input_data = torch.randn(input_shape)

# Convert the model to a TorchScript module
scripted_model = torch.jit.trace(lprnet, input_data)

# Define the input shapes for TVM
input_name = "data"
shape_list = [(input_name, input_shape)]

# Convert PyTorch model to Relay (high-level IR in TVM)
mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

# Compile to TensorIR (low-level IR in TVM)
target = "llvm"
log_file = "tuning_log.json"

print("Extract tasks...")
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)

for idx, task in enumerate(tasks):
    print("========== Task %d  (workload key: %s) ==========" % (idx, task.workload_key))
    print(task.compute_dag)


<ipython-input-26-1b2fa640db0f>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load('/content/LPRNet_Optimization/weights/Final_LPRNet_model.p

Extract tasks...
========== Task 0  (workload key: ["82c9323a6bf75f7be33236a5c2cf52cb", [1, 1, 128, 20, 90], [1, 1, 64, 18, 44]]) ==========
p0 = PLACEHOLDER [1, 1, 128, 20, 90]
pool_max(ax0, ax1, ax2, ax3, ax4) max= p0[ax0, ax1, ((ax2*2) + rv0), (ax3 + rv1), ((ax4*2) + rv2)]

========== Task 1  (workload key: ["ddfc1ebe8806378ca6f78e8a1f6be501", [1, 16, 20, 90, 4], [8, 16, 1, 1, 4, 4], [1, 8, 1, 1, 4], [1, 8, 20, 90, 4]]) ==========
p0 = PLACEHOLDER [1, 16, 20, 90, 4]
p1 = PLACEHOLDER [8, 16, 1, 1, 4, 4]
conv2d_NCHWc(n, oc_chunk, oh, ow, oc_block) += (p0[n, floordiv(ic, 4), (oh + kh), (ow + kw), floormod(ic, 4)]*p1[oc_chunk, floordiv(ic, 4), kh, kw, floormod(ic, 4), oc_block])
p2 = PLACEHOLDER [1, 8, 1, 1, 4]
T_add(ax0, ax1, ax2, ax3, ax4) = (conv2d_NCHWc[ax0, ax1, ax2, ax3, ax4] + p2[ax0, ax1, 0, 0, ax4])
T_relu(ax0, ax1, ax2, ax3, ax4) = max(T_add[ax0, ax1, ax2, ax3, ax4], 0f)

========== Task 2  (workload key: ["58bc26dbe1c1ee728a7795a35ebc9164", [1, 64, 18, 44, 4], [1, 64, 4, 18, 

In [ ]:
tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
tune_option = auto_scheduler.TuningOptions(
    num_measure_trials=200,  # change this to 20000 to achieve the best performance
    runner=auto_scheduler.LocalRunner(repeat=10, enable_cpu_cache_flush=True),
    measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
)

tuner.tune(tune_option)

|  ID  |                       Task Description                        | Latency (ms) | Speed (GFLOPS) | Trials |
-----------------------------------------------------------------------------------------------------------------
|    0 |                                  vm_mod_fused_nn_max_pool3d_1 |            - |              - |      0 |
|    1 |              vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu |            - |              - |      0 |
|    2 |                                  vm_mod_fused_nn_avg_pool2d_2 |            - |              - |      0 |
|    3 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_5 |            - |              - |      0 |
|    4 |        vm_mod_fused_nn_contrib_conv2d_NCHWc_add_add_nn_relu_1 |            - |              - |      0 |
|    5 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_1 |            - |              - |      0 |
|    6 |                                           vm_mod_fused_mean_4 |            - | 

## Key Types of Optimizations:

**Operator Fusion**: Combining multiple operations (e.g., convolution + bias add + relu, or sum + divide for mean) into a single kernel to reduce memory traffic.

**Data Layout Transformation (NCHWc)**: Changing the tensor layout to improve memory locality, enable vectorization, and better utilize hardware capabilities.

**Loop-Level Optimizations (Tiling, Vectorization, Parallelization)**:
Tiling: Dividing loops into smaller blocks to improve cache locality.

**Vectorization**: Using SIMD instructions for parallel processing of multiple data elements simultaneously.

**Parallelization**: Distributing computation across multiple CPU cores or GPU threads.

**Reduction Optimization**: Efficiently handling pooling and mean (averaging) operations by fusing reductions and using tiling/vectorization for speed.

In [ ]:
import numpy as np
# Compile with the history best
print("Compile...")
with auto_scheduler.ApplyHistoryBest(log_file):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

# Create graph executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))
data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype("float32"))
module.set_input("data", data_tvm)

# Evaluate
print("Evaluate inference time cost...")
print(module.benchmark(dev, repeat=3, min_repeat_ms=500))


Compile...
Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  34.3033      34.4002      34.6132      33.8965       0.3005                  


## Dummy Inference test

In [ ]:
# Compare original and optimized models
import time

def pytorch_inference(model, input_data, num_runs=100):
    model.eval()
    start_time = time.time()
    for _ in range(num_runs):
        with torch.no_grad():
            _ = model(input_data)
    end_time = time.time()
    return (end_time - start_time) / num_runs

def tvm_inference(lib, dev, input_data, num_runs=100):
    module = tvm.contrib.graph_executor.GraphModule(lib["default"](dev))
    module.set_input(input_name, tvm.nd.array(input_data.numpy()))
    start_time = time.time()
    for _ in range(num_runs):
        module.run()
        _ = module.get_output(0)
    end_time = time.time()
    return (end_time - start_time) / num_runs

# Run comparisons
pytorch_time = pytorch_inference(model, input_data)
dev = tvm.cpu(0)
tvm_time = tvm_inference(lib, dev, input_data)

print(f"PyTorch average inference time: {pytorch_time:.6f} seconds")
print(f"TVM optimized average inference time: {tvm_time:.6f} seconds")
print(f"Speedup: {pytorch_time / tvm_time:.2f}x")

PyTorch average inference time: 0.269682 seconds
TVM optimized average inference time: 0.046423 seconds
Speedup: 5.81x


## Model performance on Test data

In [ ]:
import time
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os
def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)


def test(model_type):
  test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
  datasets = LPRDataLoader(test_img_dirs.split(','), [94, 24], 8)

  epoch_size = len(datasets) // 1
  batch_iterator = iter(DataLoader(datasets, 1, shuffle=True, num_workers=8, collate_fn=collate_fn))
  Tp = 0
  Tn_1 = 0
  Tn_2 = 0
  t1 = time.time()
  for i in range(epoch_size):
      # load train data
      images, labels, lengths = next(batch_iterator)
      start = 0
      targets = []
      for length in lengths:
          label = labels[start:start+length]
          targets.append(label)
          start += length
      targets = np.array([el.numpy() for el in targets])
      imgs = images.numpy().copy()

      images = Variable(images)
      if model_type == 'pytorch':
        prebs = quantized_model(images)
        prebs = prebs.cpu().detach().numpy()
      else:
        module.set_input("data", tvm.nd.array(images.numpy()))

        # Run inference
        module.run()

        # Get output
        prebs = module.get_output(0).asnumpy()
      # greedy decode
      preb_labels = list()
      for i in range(prebs.shape[0]):
          preb = prebs[i, :, :]
          preb_label = list()
          for j in range(preb.shape[1]):
              preb_label.append(np.argmax(preb[:, j], axis=0))
          no_repeat_blank_label = list()
          pre_c = preb_label[0]
          if pre_c != len(CHARS) - 1:
              no_repeat_blank_label.append(pre_c)
          for c in preb_label: # dropout repeate label and blank label
              if (pre_c == c) or (c == len(CHARS) - 1):
                  if c == len(CHARS) - 1:
                      pre_c = c
                  continue
              no_repeat_blank_label.append(c)
              pre_c = c
          preb_labels.append(no_repeat_blank_label)
      for i, label in enumerate(preb_labels):
          if len(label) != len(targets[i]):
              Tn_1 += 1
              continue
          if (np.asarray(targets[i]) == np.asarray(label)).all():
              Tp += 1
          else:
              Tn_2 += 1
  Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
  print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
  t2 = time.time()
  print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

In [ ]:
test(model_type= "tvm")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Info] Test Accuracy: 0.894 [894:69:37:1000]
[Info] Test Speed: 0.049105304479599s 1/1000]
